<a href="https://colab.research.google.com/github/zhiyuan-95/Amex-default-prediction-/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=0jWnnivkyhqTsLA5Mq6weimwC1juxR&prompt=consent&access_type=offline&code_challenge=LdY27I9ieF-qYu1JBJf_eL-CwpSPjaa8UA7IjMMwlYw&code_challenge_method=S256

Enter authorization code: 4/0AZEOvhVm4P7IUnLtHuhU834ox-mK7Lp08ez2hq9N5edUqdzxoDMKO4YKNvcFxniEV1gxbQ

You are now logged in as [zhiyuan.jin1201@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID

In [ ]:
!gcloud config set project bda-12345
!gcloud config set compute/region us-west1
!gcloud config set compute/zone us-west1-a
!gcloud config set dataproc/region us-west1

Updated property [core/project].
Updated property [compute/region].
Updated property [compute/zone].
Updated property [dataproc/region].


In [ ]:
!gcloud dataproc clusters create sample --enable-component-gateway --region us-west1 --zone us-west1-a --master-machine-type n1-standard-4 --master-boot-disk-size 500 --num-workers 2 --worker-machine-type n1-standard-4 --worker-boot-disk-size 500 --image-version 2.0-debian10 --project bda-12345

Waiting on operation [projects/bda-12345/regions/us-west1/operations/af9add4f-4957-3b84-b5be-7dd9eb500eea].

Created [https://dataproc.googleapis.com/v1/projects/bda-12345/regions/us-west1/clusters/sample] Cluster placed in zone [us-west1-a].


In [ ]:
%%writefile sample.py
import itertools
import pyspark
from pyspark import SparkContext
sc = SparkContext()

header = sc.textFile('gs://aedp/expanded_data/expanded_data/part-00001').take(1)[0]
train = sc.textFile('gs://aedp/expanded_data/expanded_data/part-*')\
  .sample(False, 0.05)\
  .coalesce(1)\
  .mapPartitions(lambda x: itertools.chain([header], x))

test = sc.textFile('gs://aedp/expanded_data/expanded_data/part-*')\
          .subtract(train)\
          .sample(False, 0.2)\
          .coalesce(1)\
          .mapPartitions(lambda x: itertools.chain([header], x))

train.saveAsTextFile('gs://aedp/expanded_data/sample_train')
test.saveAsTextFile('gs://aedp/expanded_data/sample_test')

Writing sample.py


In [ ]:
!gcloud dataproc jobs submit pyspark --cluster sample sample.py

Job [837292a1e52040fca80ef4a05630b582] submitted.
Waiting for job output...
23/07/17 23:05:04 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/07/17 23:05:04 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/07/17 23:05:04 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/07/17 23:05:04 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
23/07/17 23:05:04 INFO org.sparkproject.jetty.util.log: Logging initialized @3465ms to org.sparkproject.jetty.util.log.Slf4jLog
23/07/17 23:05:04 INFO org.sparkproject.jetty.server.Server: jetty-9.4.40.v20210413; built: 2021-04-13T20:42:42.668Z; git: b881a572662e1943a14ae12e7e1207989f218b74; jvm 1.8.0_372-b07
23/07/17 23:05:04 INFO org.sparkproject.jetty.server.Server: Started @3595ms
23/07/17 23:05:04 INFO org.sparkproject.jetty.server.AbstractConnector: Started ServerConnector@8510ef2{HTTP/1.1, (http/1.1)}{0.0.0.0:39517}
23/07/17 23:05:05 INFO org.apache.hadoop.yarn.client.RMProx

In [ ]:
!gcloud dataproc clusters delete sample

The cluster 'sample' and all attached disks will be deleted.

Do you want to continue (Y/n)?  Y

Waiting on operation [projects/bda-12345/regions/us-west1/operations/95d9c955-ea9b-3470-92ea-7e3b7ce2a063].
Deleted [https://dataproc.googleapis.com/v1/projects/bda-12345/regions/us-west1/clusters/sample].


In [ ]:
%%writefile split_test.py
import itertools
import pyspark
from pyspark import SparkContext
sc = SparkContext()

header = sc.textFile('gs://aedp/expanded_data/expanded_data/part-00001').take(1)[0]
train = sc.textFile('gs://aedp/expanded_data/expanded_data/part-*')\
  .sample(False, 0.5)\
  .coalesce(1)\
  .mapPartitions(lambda x: itertools.chain([header], x))

test = sc.textFile('gs://aedp/expanded_data/expanded_data/part-*')\
          .subtract(train)\
          .sample(False, 0.5)\
          .coalesce(1)\
          .mapPartitions(lambda x: itertools.chain([header], x))

train.saveAsTextFile('gs://aedp/expanded_data/sample_train')
test.saveAsTextFile('gs://aedp/expanded_data/sample_test')